In [4]:
#分布式系统与大数据管理课程期末作业
#建立SparkSession,SparkSession是读取数据、处理元数据、配置会话和管理集群资源的入口
import os
import sys
os.environ['SPARK_HOME'] = "/data/soft/spark/"
sys.path.append("/data/soft/spark//python")
sys.path.append("/data/soft/spark//python/lib/py4j-0.10.7-src.zip")
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("finalProject").getOrCreate()
sc=spark.sparkContext

In [5]:
#第一题
#从HDFS读取数据集PM25city,存到2个分区
dataFileRDD = sc.textFile("hdfs://192.168.142.131:9000/finalProject/PM25city",2)

In [6]:
dataFileRDD.take(3)

['站号,经度,纬度,PM25,PM10,NO2,SO2,O3-1,O3-8h,CO,AQI,等级,year,month,day,hour,city',
 '99000,115.49,38.88,43,68,21,20,104,104,0.6,60,2,2018,8,1,0,北京',
 '99001,115.51,38.88,38,58,26,20,120,120,0.6,54,2,2018,8,1,0,北京']

In [5]:
#检查数据是否有重复的数据
print('Count of rows:{0}'.format(dataFileRDD.count()))
print('Count of distinct rows:{0}'.format(dataFileRDD.distinct().count()))

Count of rows:398356
Count of distinct rows:398356


In [7]:
#检查城市和PM2.5字段的缺失情况
print('Count of miss PM2.5:{0}'.format(dataFileRDD.filter(lambda row: row[3] == '').count()))
print('Count of miss city:{0}'.format(dataFileRDD.filter(lambda row: row[16] == '').count()))

Count of miss PM2.5:0
Count of miss city:0


In [23]:
#从初始RDD提取信息，提取后以numpy数组形式表示，并返回
#在此引入re模块
#纯python的效率低，因为Spark需要在python解释器和JVM之间连续切换
def extractInformation(row):
    import re
    import numpy as np

    selected_indices = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
    try:
        rs = np.array(re.split(',',row))[selected_indices]
    except:
        rs = np.array(['-99'] * len(selected_indices))
    return rs

#map（）进行转换，挨个处理每行数据,使用take()而不是collect()来查看结果，collect()返回整个RDD，take()返回单个数据分区的前n行
dataFileRDDConv = dataFileRDD.map(extractInformation)
dataFileRDDConv.take(3)

[array(['站号', '经度', '纬度', 'PM25', 'PM10', 'NO2', 'SO2', 'O3-1', 'O3-8h',
        'CO', 'AQI', '等级', 'year', 'month', 'day', 'hour', 'city'],
       dtype='<U5'),
 array(['99000', '115.49', '38.88', '43', '68', '21', '20', '104', '104',
        '0.6', '60', '2', '2018', '8', '1', '0', '北京'], dtype='<U6'),
 array(['99001', '115.51', '38.88', '38', '58', '26', '20', '120', '120',
        '0.6', '54', '2', '2018', '8', '1', '0', '北京'], dtype='<U6')]

In [24]:
#查看是不是只有第一行是字段表达意思
print('Count of useful data PM2.5:{0}'.format(dataFileRDDConv.filter(lambda row: row[3] != 'PM25').count()))
print('Count of useful data City:{0}'.format(dataFileRDDConv.filter(lambda row: row[16] != 'city').count()))

Count of useful data PM2.5:398355
Count of useful data City:398355


In [36]:
#取出城市和PM25
dataFileRDDConvPM25City = dataFileRDDConv.filter(lambda row : row[0] != '站号').map(lambda row: (row[16],float(row[3])))

In [37]:
print(dataFileRDDConvPM25City.count())
dataFileRDDConvPM25City.take(3)

398355


[('北京', 43.0), ('北京', 38.0), ('北京', 50.0)]

In [38]:
#求各城市的PM25最大值
cityPM25Max = dataFileRDDConvPM25City.reduceByKey(max)
cityPM25Max.collect()

[('天津', 589.0),
 ('济南', 534.0),
 ('成都', 359.0),
 ('呼和浩特', 530.0),
 ('昆明', 288.0),
 ('北京', 589.0),
 ('上海', 288.0),
 ('青岛', 425.0),
 ('厦门', 303.0),
 ('郑州', 521.0),
 ('乌鲁木齐', 546.0),
 ('海口', 167.0)]

In [41]:
#求各城市的PM25最大值
cityPM25Min = dataFileRDDConvPM25City.reduceByKey(min)
cityPM25Min.collect()

[('天津', 0.0),
 ('济南', 0.0),
 ('成都', 0.0),
 ('呼和浩特', 0.0),
 ('昆明', 0.0),
 ('北京', 0.0),
 ('上海', 0.0),
 ('青岛', 0.0),
 ('厦门', 0.0),
 ('郑州', 0.0),
 ('乌鲁木齐', 0.0),
 ('海口', 0.0)]

In [48]:
#求各城市的PM25最大值
cityPM25Count = dataFileRDDConvPM25City.countByKey().items()


In [49]:
print(cityPM25Count)

dict_items([('北京', 37628), ('上海', 55441), ('天津', 55384), ('青岛', 39335), ('济南', 37609), ('厦门', 22304), ('郑州', 54302), ('乌鲁木齐', 14838), ('成都', 41165), ('呼和浩特', 13728), ('海口', 9398), ('昆明', 17223)])


In [50]:
cityPM25Min = dataFileRDDConvPM25City.groupByKey().mapValues(list).map(lambda x:(x[0],sum(x[1])/len(x[1])))
cityPM25Average.collect()

NameError: name 'cityPM25Average' is not defined

In [19]:
#求各城市PM25平均值
cityPM25Average = dataFileRDDConvPM25City.groupByKey().mapValues(list).map(lambda x:(x[0],sum(x[1])/len(x[1])))
cityPM25Average.collect()

[('天津', 56.37377220858009),
 ('济南', 83.21077401685767),
 ('成都', 56.21107737155351),
 ('呼和浩特', 41.75553613053613),
 ('昆明', 23.64524182778842),
 ('北京', 54.73764218135431),
 ('上海', 41.380602802979745),
 ('青岛', 60.41360111859667),
 ('厦门', 27.681850789096128),
 ('郑州', 94.13384405730912),
 ('乌鲁木齐', 83.67879768162825),
 ('海口', 16.023941264098745)]

In [51]:
#第二题 首先建立DataFrame 
#指定DataFrame的结构
import pyspark.sql.types as typ
labels = [
    ("站号",typ.StringType()), ("经度",typ.StringType()), ("纬度",typ.StringType()),
    ("PM25",typ.FloatType()),  ("PM10",typ.FloatType()),  ("NO2",typ.FloatType()),
    ("SO2",typ.FloatType()),   ("O3-1",typ.FloatType()),  ("O3-8h",typ.FloatType()),
    ("CO",typ.FloatType()),    ("AQI",typ.FloatType()),   ("等级",typ.FloatType()),
    ("year",typ.StringType()), ("month",typ.StringType()),("day",typ.StringType()),
    ("hour",typ.StringType()), ("city",typ.StringType())]
schema = typ.StructType([
        typ.StructField(e[0], e[1], False) for e in labels])

In [52]:
#从hdfs文件生成DataFrame
dataFileDF = spark.read.csv("hdfs://192.168.142.131:9000/finalProject/PM25city",header=True,schema=schema,sep=',')
dataFileDF.count()

398355

In [53]:
dataFileDF.show(5)

+-----+------+-----+----+-----+----+----+-----+-----+---+----+---+----+-----+---+----+----+
|   站号|    经度|   纬度|PM25| PM10| NO2| SO2| O3-1|O3-8h| CO| AQI| 等级|year|month|day|hour|city|
+-----+------+-----+----+-----+----+----+-----+-----+---+----+---+----+-----+---+----+----+
|99000|115.49|38.88|43.0| 68.0|21.0|20.0|104.0|104.0|0.6|60.0|2.0|2018|    8|  1|   0|  北京|
|99001|115.51|38.88|38.0| 58.0|26.0|20.0|120.0|120.0|0.6|54.0|2.0|2018|    8|  1|   0|  北京|
|99002|115.47|38.91|50.0| 72.0|22.0|17.0|113.0|113.0|0.7|69.0|2.0|2018|    8|  1|   0|  北京|
|99004|115.45|38.88|52.0|108.0|46.0|21.0| 66.0| 66.0|0.7|79.0|2.0|2018|    8|  1|   0|  北京|
|99006|116.36|39.87|55.0| 80.0|27.0| 2.0| 60.0| 60.0|1.3|75.0|2.0|2018|    8|  1|   0|  北京|
+-----+------+-----+----+-----+----+----+-----+-----+---+----+---+----+-----+---+----+----+
only showing top 5 rows



In [54]:
#创建临时视图以便适应spark SQL
dataFileDF.createOrReplaceTempView("dataFileDFView")

In [55]:
spark.sql("select * from dataFileDFView where city = '成都'").show(5)

+-----+------+-----+----+----+-----+---+----+-----+---+----+---+----+-----+---+----+----+
|   站号|    经度|   纬度|PM25|PM10|  NO2|SO2|O3-1|O3-8h| CO| AQI| 等级|year|month|day|hour|city|
+-----+------+-----+----+----+-----+---+----+-----+---+----+---+----+-----+---+----+----+
|99052| 104.0|30.72|28.0|38.0| 72.0|4.0| 1.0|  1.0|1.2|40.0|1.0|2018|    8|  1|   0|  成都|
|99053|104.19|30.67|41.0|79.0|120.0|8.0| 5.0|  5.0|1.2|65.0|2.0|2018|    8|  1|   0|  成都|
|99054|104.08| 30.6|27.0|46.0| 87.0|2.0| 1.0|  1.0|0.7|46.0|1.0|2018|    8|  1|   0|  成都|
|99055|104.12|30.63|37.0|50.0| 61.0|9.0|24.0| 24.0|1.1|53.0|2.0|2018|    8|  1|   0|  成都|
|98537|104.06|30.66|56.0|89.0| 94.0|5.0|15.0| 15.0|1.2|77.0|2.0|2018|    8|  1|   0|  成都|
+-----+------+-----+----+----+-----+---+----+-----+---+----+---+----+-----+---+----+----+
only showing top 5 rows



In [56]:
dataFileDF.printSchema()

root
 |-- 站号: string (nullable = true)
 |-- 经度: string (nullable = true)
 |-- 纬度: string (nullable = true)
 |-- PM25: float (nullable = true)
 |-- PM10: float (nullable = true)
 |-- NO2: float (nullable = true)
 |-- SO2: float (nullable = true)
 |-- O3-1: float (nullable = true)
 |-- O3-8h: float (nullable = true)
 |-- CO: float (nullable = true)
 |-- AQI: float (nullable = true)
 |-- 等级: float (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- city: string (nullable = true)



In [57]:
#北京在2019.02.01这一天的数据量
spark.sql("select count(*) from dataFileDFView where city = '北京' and year = '2019' and month = '2' and day = '1'").show()

+--------+
|count(1)|
+--------+
|     527|
+--------+



In [58]:
#获取北京201902数据，一共28天
data_201902_beijing=spark.sql("select avg(AQI) as AQI,int(day) as day from dataFileDFView \
where year == '2019' and month == '2' and city == '北京' group by day")

In [59]:
#查询数据有多少月份
data_month=spark.sql("select distinct (year,month) from dataFileDFView")
data_month.show()

+--------------------------------------+
|named_struct(year, year, month, month)|
+--------------------------------------+
|                             [2018, 8]|
|                             [2019, 6]|
|                             [2019, 2]|
|                             [2019, 1]|
+--------------------------------------+



In [60]:
sys.version

'3.7.3 (default, Mar 27 2019, 22:11:17) \n[GCC 7.3.0]'

In [61]:
data_201902_beijing.count()

28

In [62]:
data_201902_beijing.orderBy('day').collect()

[Row(AQI=86.70208728652752, day=1),
 Row(AQI=143.92638036809817, day=2),
 Row(AQI=121.0806142034549, day=3),
 Row(AQI=99.8956043956044, day=4),
 Row(AQI=170.7734082397004, day=5),
 Row(AQI=78.51948051948052, day=6),
 Row(AQI=37.265469061876246, day=7),
 Row(AQI=46.540117416829744, day=8),
 Row(AQI=31.506276150627617, day=9),
 Row(AQI=51.72357723577236, day=10),
 Row(AQI=83.689453125, day=11),
 Row(AQI=66.37869822485207, day=12),
 Row(AQI=51.111617312072894, day=13),
 Row(AQI=64.50769230769231, day=14),
 Row(AQI=37.24455205811138, day=15),
 Row(AQI=27.83622350674374, day=16),
 Row(AQI=35.91575091575091, day=17),
 Row(AQI=77.89781021897811, day=18),
 Row(AQI=156.68464730290455, day=19),
 Row(AQI=125.2179104477612, day=20),
 Row(AQI=169.34751773049646, day=21),
 Row(AQI=207.9163179916318, day=22),
 Row(AQI=195.65865384615384, day=23),
 Row(AQI=158.76923076923077, day=24),
 Row(AQI=75.21256038647343, day=25),
 Row(AQI=66.73849372384937, day=26),
 Row(AQI=101.7125, day=27),
 Row(AQI=142.022

In [63]:
data_201902_beijing.createOrReplaceTempView("data_201902_beijingView")
spark.sql("select * from data_201902_beijingView where AQI <= 50").count()


6

In [64]:
spark.sql("select count(*) from data_201902_beijingView where AQI > 50 and AQI <= 100").show()


+--------+
|count(1)|
+--------+
|      11|
+--------+



In [65]:
spark.sql("select count(*) from data_201902_beijingView where AQI > 100 and AQI <=150").show()


+--------+
|count(1)|
+--------+
|       5|
+--------+



In [66]:
spark.sql("select count(*) from data_201902_beijingView where AQI > 151 and AQI <= 200").show()


+--------+
|count(1)|
+--------+
|       5|
+--------+



In [67]:
spark.sql("select count(*) from data_201902_beijingView where AQI > 200").show()

+--------+
|count(1)|
+--------+
|       1|
+--------+



In [68]:
#获取上海201902数据，一共28天
data_201902_shanghai=spark.sql("select avg(AQI) as AQI,int(day) as day from dataFileDFView \
where year == '2019' and month == '2' and city == '上海' group by day")
data_201902_shanghai.orderBy('day').collect()

[Row(AQI=87.12014134275618, day=1),
 Row(AQI=82.09510086455332, day=2),
 Row(AQI=99.23975409836065, day=3),
 Row(AQI=136.88491048593352, day=4),
 Row(AQI=70.6678486997636, day=5),
 Row(AQI=85.75257731958763, day=6),
 Row(AQI=54.767100977198695, day=7),
 Row(AQI=28.922365988909426, day=8),
 Row(AQI=31.957303370786516, day=9),
 Row(AQI=30.294117647058822, day=10),
 Row(AQI=44.08303249097473, day=11),
 Row(AQI=45.11092985318108, day=12),
 Row(AQI=44.024566473988436, day=13),
 Row(AQI=45.70854271356784, day=14),
 Row(AQI=28.83941605839416, day=15),
 Row(AQI=63.211717709720375, day=16),
 Row(AQI=41.034979423868315, day=17),
 Row(AQI=29.666011787819254, day=18),
 Row(AQI=42.13573407202216, day=19),
 Row(AQI=80.91749174917491, day=20),
 Row(AQI=85.51724137931035, day=21),
 Row(AQI=77.63254593175853, day=22),
 Row(AQI=161.39407744874714, day=23),
 Row(AQI=154.33130081300814, day=24),
 Row(AQI=116.38689217758986, day=25),
 Row(AQI=63.329351535836174, day=26),
 Row(AQI=44.53884711779449, day=27)

In [77]:
data_201902_shanghai.createOrReplaceTempView("data_201902_shanghaiView")
spark.sql("select * from data_201902_shanghaiView where AQI <= 50").count()


12

In [78]:
spark.sql("select count(*) from data_201902_shanghaiView where AQI > 50 and AQI <= 100").show()

+--------+
|count(1)|
+--------+
|      12|
+--------+



In [79]:
spark.sql("select count(*) from data_201902_shanghaiView where AQI > 100 and AQI <= 150").show()

+--------+
|count(1)|
+--------+
|       2|
+--------+



In [80]:
spark.sql("select count(*) from data_201902_shanghaiView where AQI > 150 and AQI <= 200").show()

+--------+
|count(1)|
+--------+
|       2|
+--------+



In [81]:
spark.sql("select count(*) from data_201902_shanghaiView where AQI > 200 ").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [83]:
data_201902_chengdu=spark.sql("select avg(AQI) as AQI,int(day) as day from dataFileDFView where year == '2019' and month == '2' and city == '成都' group by day")

In [84]:
data_201902_chengdu.orderBy('day').collect()

[Row(AQI=85.16606498194946, day=1),
 Row(AQI=94.64737793851718, day=2),
 Row(AQI=72.72744360902256, day=3),
 Row(AQI=80.15724381625442, day=4),
 Row(AQI=149.9373913043478, day=5),
 Row(AQI=104.1522491349481, day=6),
 Row(AQI=85.05602716468591, day=7),
 Row(AQI=75.45640074211502, day=8),
 Row(AQI=56.70477815699659, day=9),
 Row(AQI=43.973333333333336, day=10),
 Row(AQI=53.89087656529517, day=11),
 Row(AQI=52.68352059925093, day=12),
 Row(AQI=74.76785714285714, day=13),
 Row(AQI=82.96625222024866, day=14),
 Row(AQI=70.23255813953489, day=15),
 Row(AQI=67.88945578231292, day=16),
 Row(AQI=55.329446064139944, day=17),
 Row(AQI=37.37623762376238, day=18),
 Row(AQI=57.574585635359114, day=19),
 Row(AQI=76.46460980036298, day=20),
 Row(AQI=73.71327433628319, day=21),
 Row(AQI=110.3632958801498, day=22),
 Row(AQI=117.37912087912088, day=23),
 Row(AQI=69.85191637630662, day=24),
 Row(AQI=73.10990990990992, day=25),
 Row(AQI=65.93320610687023, day=26),
 Row(AQI=58.45420560747664, day=27),
 Row(A

In [85]:
data_201902_chengdu.createOrReplaceTempView("data_201902_chengduView")
spark.sql("select count(*) from data_201902_chengduView where AQI <= 50").show()

+--------+
|count(1)|
+--------+
|       2|
+--------+



In [86]:
spark.sql("select count(*) from data_201902_chengduView where AQI > 50 and AQI <= 100").show()

+--------+
|count(1)|
+--------+
|      22|
+--------+



In [87]:
spark.sql("select count(*) from data_201902_chengduView where AQI > 100 and AQI <= 150").show()

+--------+
|count(1)|
+--------+
|       4|
+--------+



In [88]:
spark.sql("select count(*) from data_201902_chengduView where AQI > 150 and AQI <= 200").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [89]:
spark.sql("select count(*) from data_201902_chengduView where AQI > 200 ").show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [69]:
#第三题 利用PySpark的ML包对成都的PM25进行回归分析
dataFileDFChengDuRegPm25 =spark.sql("select PM25 as PM25,PM10 as PM10,NO2 as NO2,SO2 as SO2,CO as CO from dataFileDFView where city = '成都'") 

In [70]:
dataFileDFChengDuRegPm25.count()

41165

In [71]:
dataFileDFChengDuRegPm25.show(10)

+----+----+-----+----+---+
|PM25|PM10|  NO2| SO2| CO|
+----+----+-----+----+---+
|28.0|38.0| 72.0| 4.0|1.2|
|41.0|79.0|120.0| 8.0|1.2|
|27.0|46.0| 87.0| 2.0|0.7|
|37.0|50.0| 61.0| 9.0|1.1|
|56.0|89.0| 94.0| 5.0|1.2|
|16.0|25.0| 16.0| 8.0|0.9|
|26.0|72.0| 65.0|14.0|1.0|
|14.0|32.0| 53.0| 8.0|0.8|
|28.0|54.0| 29.0| 5.0|0.6|
| 5.0|14.0|  7.0| 9.0|0.4|
+----+----+-----+----+---+
only showing top 10 rows



In [72]:
#排除离群值
cols = ['PM25','PM10','NO2','SO2','CO']
bounds = {}

for col in cols:
    quantiles = dataFileDFChengDuRegPm25.approxQuantile(col, [0.25, 0.75], 0.05)
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [quantiles[0] - 1.5 * IQR, quantiles[1] + 1.5 * IQR]

In [73]:
bounds

{'PM25': [-21.5, 126.5],
 'PM10': [-18.0, 174.0],
 'NO2': [-25.5, 90.5],
 'SO2': [0.5, 12.5],
 'CO': [0.2499999701976776, 1.4500000178813934]}

In [74]:
dataFileDFChengDuRegPm25.printSchema()

root
 |-- PM25: float (nullable = true)
 |-- PM10: float (nullable = true)
 |-- NO2: float (nullable = true)
 |-- SO2: float (nullable = true)
 |-- CO: float (nullable = true)



In [87]:
#检查PM25与PM10的关系
from pyspark.sql.functions import corr
dataFileDFChengDuRegPm25.select(corr('PM25','PM10')).show()

+-----------------+
| corr(PM25, PM10)|
+-----------------+
|0.930331337950205|
+-----------------+



In [88]:
from pyspark.sql.functions import corr
dataFileDFChengDuRegPm25.select(corr('PM25','NO2')).show()

+------------------+
|   corr(PM25, NO2)|
+------------------+
|0.4764789366268503|
+------------------+



In [89]:
from pyspark.sql.functions import corr
dataFileDFChengDuRegPm25.select(corr('PM25','SO2')).show()

+-------------------+
|    corr(PM25, SO2)|
+-------------------+
|0.13638977091407592|
+-------------------+



In [90]:
from pyspark.sql.functions import corr
dataFileDFChengDuRegPm25.select(corr('PM25','CO')).show()

+------------------+
|    corr(PM25, CO)|
+------------------+
|0.5110549464395842|
+------------------+



In [107]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler
vec = VectorAssembler(inputCols=['PM10','NO2','SO2','CO'],outputCol = 'features')

In [108]:
features_df = vec.transform(dataFileDFChengDuRegPm25)

In [109]:
features_df.printSchema()

root
 |-- PM25: float (nullable = true)
 |-- PM10: float (nullable = true)
 |-- NO2: float (nullable = true)
 |-- SO2: float (nullable = true)
 |-- CO: float (nullable = true)
 |-- features: vector (nullable = true)



In [110]:
model_df = features_df.select('features','PM25')
model_df.show(5)

+--------------------+----+
|            features|PM25|
+--------------------+----+
|[38.0,72.0,4.0,1....|28.0|
|[79.0,120.0,8.0,1...|41.0|
|[46.0,87.0,2.0,0....|27.0|
|[50.0,61.0,9.0,1....|37.0|
|[89.0,94.0,5.0,1....|56.0|
+--------------------+----+
only showing top 5 rows



In [111]:
model_df.count()

41165

In [112]:
#划分数据集
train_df,test_df = model_df.randomSplit([0.7,0.3])
print((train_df.count(),len(train_df.columns)))

(28839, 2)


In [117]:
from pyspark.ml.regression import LinearRegression
lin_reg = LinearRegression(labelCol='PM25')
lr_model = lin_reg.fit(train_df)
lr_model.coefficients

DenseVector([0.7009, -0.0673, -0.5948, 6.9726])

In [118]:
lr_model.intercept

-1.7874492318242212

In [119]:
#在测试数据上评估模型
test_p = lr_model.evaluate(test_df)
test_p.r2

0.8743725638646

In [120]:
test_p.meanSquaredError

129.7138600792581